In [1]:
from IPython.display import HTML
display(HTML("<head><link rel='stylesheet' type='text/css' href='./../../static/custom.css'></head>"))
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
from bqplot import *
import bqplot as bq
import bqplot.marks as bqm
import bqplot.scales as bqs
import bqplot.axes as bqa

import ipywidgets as widgets

In [3]:
def calculate_critic(a, b):
    
    """
        This function calculates the critic point 
        (p_c, v_c, T_c) from given a and b parameters of 
        the Van der Waals equation of state for real gases.
        
        :math:`(P + a \\frac{n^2}{V^2})(V - nb) = nRT`
        
        :math:`p_c = \\frac{a}{27 b^2}`
        :math:`v_c = 3b`
        :math:`T_c = \\frac{8a}{27 b R}`
        
   Args:
       a: Term related with the attraction between particles in
       L^2 bar/mol^2.\n
       b: Term related with the volume that is occupied by one 
       mole of the molecules in L/mol.\n
       
   Returns:
       p_c: Critical pressure in bar.\n
       v_c: Critical volume in L/mol.\n
       T_c: Critical tenperature in K.\n
        
    """
    
    if b == 0.0:
        return None
    
    k_B = 1.3806488e-23 #m^2 kg s^-2 K^-1
    N_A = 6.02214129e23 
    R = 0.082 * 1.01325 #atm L mol^-1 K^-1 #now in bar L mol^-1 K^-1
    
    p_c = a/27.0/(b**2)
    v_c = 3.0*b
    T_c = 8.0*a/27.0/b/R
    
    return p_c, v_c, T_c

In [4]:
def get_absolute_isotherms(a, b, v_values, T_values):
    #given v (1d array) and T (Nd array) calculate N isotherm arrays 
    
    isotherms = []
    
    #N_A = 6.02214129e23 

    R = 0.082 * 1.01325 #atm L mol^-1 K^-1 #now in bar L mol^-1 K^-1

    for T in T_values:
        isot = []
        for v in v_values:
            p = R*T/(v - b) - (a/v**2)
            isot.append(p)
            
        isotherms.append(isot)
        
    return isotherms

In [5]:
def get_zoom_arrays(initial_x_range, final_x_range, initial_y_range, final_y_range, size):
    
    x_min = np.linspace(min(initial_x_range), min(final_x_range), size)
    x_max = np.linspace(max(initial_x_range), max(final_x_range), size)
    
    y_min = np.linspace(min(initial_y_range), min(final_y_range), size)
    y_max = np.linspace(max(initial_y_range), max(final_y_range), size)
    
    return x_min, x_max, y_min, y_max

In [6]:
def update_scales(change):
    
    index = change.owner.value
    scale_x_118_001.min, scale_x_118_001.max, scale_y_118_001.min, scale_y_118_001.max = x_min[index], x_max[index], y_min[index], y_max[index]

In [7]:
def bar_to_atm(p_values):
    
    for i in range(len(p_values)):
        p_values[i] = np.array(p_values[i]) * 0.9869 #unit change from bar to atm
        
    return p_values

In [13]:
def update_isotherms(change):

    
    p_values = get_absolute_isotherms(a_slider.value, b_slider.value, v_values, T_values)
    p_values = bar_to_atm(p_values)
    
    marks = [bqm.Lines(
                   x = [v_values for elem in p_values], 
                   y = p_values, 
                   scales = {'x': scale_x_119_001, 'y': scale_y_119_001}, 
                   opacities = [1.0],
                   visible = True, #True, #t == '1.00',
                   colors = colors,
                   #labels = '',
                   )]
    
    fig_119_002.marks = marks

In [15]:
#In this program we are going to use water's parameters
a = 5.536 #L^2 bar mol^-2
b = 0.03049 #L mol^-1

colors = ['#0079c4','#f09205','#21c400', '#850082']

p_c, v_c, T_c = calculate_critic(a, b)

p_c = p_c * 0.9869 #unit change from bar to atm

v_values = np.linspace(-5, 5, 3000) #L/mol
T_values = [0.9*T_c, 1.0*T_c, 1.1*T_c]

p_values = get_absolute_isotherms(a, b, v_values, T_values)
p_values = bar_to_atm(p_values)

#####################
######TOP BLOCK######
#####################

top_block_119_000 = widgets.VBox([], layout=widgets.Layout(align_items='center'))

scale_x_119_001 = bqs.LinearScale(min = -0.5, max = 0.5)
scale_y_119_001 = bqs.LinearScale(min = -300, max = 300)

axis_x_119_001 = bqa.Axis(
                    scale=scale_x_119_001,
                    tick_format='.1f',#'0.2f',
                    tick_style={'font-size': '15px'},
                    num_ticks=5,
                    grid_lines = 'none',
                    grid_color = '#8e8e8e', 
                    label='v (L/mol)',
                    label_location='middle',
                    label_style={'stroke': 'black', 'default-size': 35},
                    label_offset='50px')

axis_y_119_001 = bqa.Axis(
                scale=scale_y_119_001,
                tick_format='.0f',#'0.2f',
                tick_style={'font-size': '15px'},
                num_ticks=4,
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                orientation='vertical',
                label='p (atm)',
                label_location='middle',
                label_style={'stroke': 'red', 'default_size': 35},
                label_offset='50px')

fig_119_001 = Figure(
                title='Van der Waals equation of state',
                marks=[],
                axes=[axis_x_119_001, axis_y_119_001],
                animation_duration=0, #500,
                #layout = widgets.Layout(align_self='center', width='75%'),
                legend_location='top-right',
                background_style= {'fill': 'white',  'stroke': 'black'},
                min_aspect_ratio=1.0,
                fig_margin=dict(top=70, bottom=60, left=80, right=30),
                toolbar = True,
                layout = widgets.Layout()
                      )

marks = [bqm.Lines(
               x = [v_values for elem in p_values], 
               y = p_values, 
               scales = {'x': scale_x_119_001, 'y': scale_y_119_001}, 
               opacities = [1.0],
               visible = True, #True, #t == '1.00',
               colors = colors,
               #labels = '',
               )]

fig_119_001.marks = marks

tb_119_001 = Toolbar(figure=fig_119_001, layout=widgets.Layout(align_self='center'))

fig_119_002 = Figure(
                title='Van der Waals equation of state',
                marks=[],
                axes=[axis_x_119_001, axis_y_119_001],
                animation_duration=0, #500,
                #layout = widgets.Layout(align_self='center', width='75%'),
                legend_location='top-right',
                background_style= {'fill': 'white',  'stroke': 'black'},
                min_aspect_ratio=1.0,
                fig_margin=dict(top=70, bottom=60, left=80, right=30),
                toolbar = True,
                layout = widgets.Layout()
                      )

fig_119_002.marks = marks


top_block_119_000.children = [tb_119_001, widgets.HBox([fig_119_001, fig_119_002])]

#####################
######BOTTOM BLOCK###
#####################

bottom_block_119_000 = widgets.VBox([], layout=widgets.Layout(align_items='center', width='100%', margin='30px 0 0 0'))

a_slider = widgets.FloatSlider(
    min=0,
    max=34.0,
    step=0.001,
    value=a,
    description='a',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    layout=widgets.Layout(width='90%'),
)

a_slider.observe(update_isotherms, 'value')

b_slider = widgets.FloatSlider(
    min=0,
    max=0.1735,
    step=0.0001,
    value=b,
    description='b',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    layout=widgets.Layout(width='90%'),
)

b_slider.observe(update_isotherms, 'value')

bottom_block_119_000.children = [widgets.HBox([a_slider,
                                       widgets.HTMLMath(value=r"\( \frac{L^2 bar}{mol^2} \)",layout=widgets.Layout(height='60px') )],
                                      layout=widgets.Layout(width='50%', height='100%')),
                         widgets.HBox([b_slider,
                                       widgets.HTMLMath(value=r"\( \frac{L}{mol} \)",layout=widgets.Layout(height='60px'))],
                                      layout=widgets.Layout(width='50%', height='100%'))]
#####################
####MAIN BLOCK#####
#####################

main_block_119_000 = widgets.VBox([], layout=widgets.Layout(align_items='center'))

main_block_119_000.children = [top_block_119_000, 
                               bottom_block_119_000
                               ]

main_block_119_000